In [1]:
# !pip install numpy
# !pip install opencv-python
# !pip install tqdm
# !pip install mediapipe

In [1]:
images_in_folder = './DATASET/TRAIN'
images_out_folder = 'fitness_poses_images_out_basic'
csv_out_path = 'fitness_poses_csvs_out_basic.csv'

In [3]:
import csv
import cv2
import numpy as np
import os
import sys
import tqdm

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose

with open(csv_out_path, 'w') as csv_out_file:
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

      # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

    for pose_class_name in pose_class_names:
        print('Bootstrapping ', pose_class_name, file=sys.stderr)
        if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
            os.makedirs(os.path.join(images_out_folder, pose_class_name))
        image_names = sorted([
            n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
            if not n.startswith('.')])
        for image_name in tqdm.tqdm(image_names, position=0):
          # Load image.
          input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
        input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

          # Initialize fresh pose tracker and run it.
        with mp_pose.Pose() as pose_tracker:
            result = pose_tracker.process(image=input_frame)
            pose_landmarks = result.pose_landmarks

          # Save image with pose prediction (if pose was detected).
        output_frame = input_frame.copy()
        if pose_landmarks is not None:
            mp_drawing.draw_landmarks(
                image=output_frame,
                landmark_list=pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS)
        output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)

          # Save landmarks.
        if pose_landmarks is not None:
            # Check the number of landmarks and take pose landmarks.
            assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

            # Map pose landmarks from [0, 1] range to absolute coordinates to get
            # correct aspect ratio.
            frame_height, frame_width = output_frame.shape[:2]
            pose_landmarks *= np.array([frame_width, frame_height, frame_width])

            # Write pose sample to CSV.
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.str).tolist()
            csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)


Bootstrapping  downdog
100%|██████████| 223/223 [00:08<00:00, 26.12it/s]
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/tmp/ipykernel_9141/1768560441.py:56: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.str).tolist()
Bootstrapping  goddess
100%|██████████| 180/180 [00:12<00:00, 13.90it/s]
Bootstrapping  plank
100%|██████████| 266/266 [00:17<00:00, 14.96it/s]
Bootstrapping  tree
100%|██████████| 160/160 [00:12<00:00, 12.55it/s]
Bootstrapping  warrior2
100%|██████████| 252/252 [00:17<00:00, 14.43it/s]


In [4]:
import pandas as pd
df = pd.read_csv('./fitness_poses_csvs_out_basic.csv')
df.tail()

,00000427.jpg,downdog,173.51081,629.28033,-97.5422,162.36895,630.64472,-120.30973,161.35009,629.44993,...,-69.4363,447.44623,683.05438,139.86184,378.61522,148.83673,-198.62442,383.33168,699.25347,97.32224
0,00000401.jpg,goddess,183.13453,218.72329,-380.40924,187.59159,213.27152,-363.54706,190.06060,213.29792,...,23.32020,130.06577,409.55200,27.44759,263.75070,427.68334,-57.24016,112.37321,429.84871,-46.95151
1,00000440.jpg,plank,63.19916,77.00044,-71.20605,61.97592,72.33012,-63.98299,62.74045,71.14140,...,40.30488,231.78807,133.39856,17.12119,247.75290,129.45002,-16.60351,246.38909,132.64798,-40.27478
2,File84.jpg,tree,1089.95476,156.51797,-248.29167,1097.34844,138.42693,-208.61801,1103.74866,138.44502,...,528.58583,1115.96855,934.75349,307.68977,1080.84377,717.60290,463.83662,1109.51557,982.97223,115.05697
3,00000438.jpg,warrior2,471.19268,715.32705,123.60308,465.22788,703.90811,110.53562,461.07924,703.05372,...,257.69346,579.74907,1093.32558,148.39453,211.21090,1086.36185,251.09657,645.92125,1081.85484,142.38165
